In [21]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [22]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [23]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [24]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [25]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [26]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [27]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [28]:
# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [29]:
# Compile model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Train model
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3121 - loss: 1.5113 - val_accuracy: 0.6508 - val_loss: 1.3138
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5765 - loss: 1.2794 - val_accuracy: 0.6772 - val_loss: 1.2126
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5729 - loss: 1.2355 - val_accuracy: 0.6402 - val_loss: 1.1523
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6071 - loss: 1.1658 - val_accuracy: 0.6931 - val_loss: 1.0822
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6700 - loss: 1.0807 - val_accuracy: 0.7090 - val_loss: 1.0091
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7128 - loss: 1.0497 - val_accuracy: 0.7460 - val_loss: 0.9285
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7469 - loss: 0.9473 - val_accuracy: 0.8148 - val_loss: 0.8758
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7805 - loss: 0.8980 - val_accuracy: 0.8730 - val_loss:

In [31]:
# Save the training history
with open('training_history.json', 'w') as f:
    json.dump(lstm_history.history, f)

In [32]:
# Save LSTM model
lstm_model.save('verification_model_implementation_2.h5')